In [ ]:
import path, sys
cur_path = path.Path('.').abspath()
sys.path.append(cur_path.parent.parent)
from DRAEM.test.model_pack import DRAEMPack

model = DRAEMPack(in_channels=8)
model.load_pack_checkpoint('/home/caoxiatian/DRAEM/checkpoints/gray_ms1k/81000.ckpt')
# model = model
print(model)

In [ ]:
import torch
from nni.compression.pruning import L1NormPruner
config_list = [{
    'op_types': ['Conv2d'],
    'sparse_ratio': 0.5
}]
pruner = L1NormPruner(model, config_list)

# show the wrapped model structure, `PrunerModuleWrapper` have wrapped the layers that configured in the config_list.
print(model)

In [ ]:
# compress the model and generate the masks
_, masks = pruner.compress()
# show the masks sparsity
for name, mask in masks.items():
    print(name, ' sparsity : ', '{:.2}'.format(mask['weight'].sum() / mask['weight'].numel()))

In [ ]:
# need to unwrap the model, if the model is wrapped before speedup
pruner.unwrap_model()
# speedup the model, for more information about speedup, please refer :doc:`pruning_speedup`.
from nni.compression.speedup import ModelSpeedup

ModelSpeedup(model, torch.rand(1, 8, 640, 640), masks).speedup_model()

In [ ]:
print(model)

In [ ]:
torch.save(model, 'compressed_bubbles.mod')